# Titanic Classification

In [19]:
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


#### Reading the Dataset

In [20]:
# Load the dataset
titanic_data = pd.read_csv('./Titanic-Dataset.csv')

# Display the first few rows of the dataframe
titanic_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### checking for empty data.

In [21]:
# Check for missing values in the dataset
missing_values = titanic_data.isnull().sum()
missing_values[missing_values > 0]

Age         177
Cabin       687
Embarked      2
dtype: int64

In [22]:
# Impute 'Age' with median and 'Embarked' with most frequent value
imputer = SimpleImputer(strategy='median')
titanic_data['Age'] = imputer.fit_transform(titanic_data[['Age']])

imputer = SimpleImputer(strategy='most_frequent')
titanic_data['Embarked'] = imputer.fit_transform(titanic_data[['Embarked']])

# Dropping 'Cabin' due to high percentage of missing values
titanic_data.drop('Cabin', axis=1, inplace=True)

# Feature engineering: Creating a new feature 'FamilySize'
titanic_data['FamilySize'] = titanic_data['SibSp'] + titanic_data['Parch'] + 1

# Dropping features that are unlikely to be useful in prediction
titanic_data.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

# Separating features and target variable
X = titanic_data.drop('Survived', axis=1)
y = titanic_data['Survived']

# Encoding categorical variables and scaling numerical variables
categorical_features = ['Sex', 'Embarked']
numerical_features = ['Age', 'Fare', 'Pclass', 'SibSp', 'Parch', 'FamilySize']


##### Create the preprocessing Pipeline

In [23]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Create the model pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(random_state=42))])


##### Data fitting and evaluations

In [24]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the pipeline
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

Accuracy

In [25]:
accuracy

0.8268156424581006

Classification

In [26]:
print(classification_rep)

              precision    recall  f1-score   support

           0       0.85      0.86      0.85       105
           1       0.79      0.78      0.79        74

    accuracy                           0.83       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.83      0.83      0.83       179

